
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

<i18n value="2b5dc285-0d50-4ea7-a71b-8a7aa355ad7c"/>

# Pandas UDFを使った推論 (Inference with Pandas UDFs)

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) このレッスンでは次を行います:<br>
- scikit-learnのモデルを構築し、MLflowで追跡そしてPandas Scalar Iterator UDFsと **`mapInPandas()`** を使って大規模に適用します。

Pandas UDFについて詳しく知りたい方は、こちらの<a href="https://databricks.com/blog/2020/05/20/new-pandas-udfs-and-python-type-hints-in-the-upcoming-release-of-apache-spark-3-0.html" target="_blank">ブログ記事</a>でSpark 3.0の新機能を参照してください。

In [0]:
%run ./Includes/Classroom-Setup

<i18n value="8b52bca0-45f0-4ada-be31-c2c473fb8e77"/>

sklearnのモデルを学習し、MLflowで記録します。

In [0]:
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

with mlflow.start_run(run_name="sklearn-random-forest") as run:
    # Enable autologging 
    mlflow.sklearn.autolog(log_input_examples=True, log_model_signatures=True, log_models=True)
    # Import the data
    df = pd.read_csv(f"{DA.paths.datasets}/airbnb/sf-listings/airbnb-cleaned-mlflow.csv".replace("dbfs:/", "/dbfs/")).drop(["zipcode"], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)

    # Create model
    rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)

<i18n value="7ebcaaf9-c6f5-4c92-865a-c7f2c7afb555"/>

Spark DataFrameの作成

In [0]:
spark_df = spark.createDataFrame(X_test)

<i18n value="1cdc4475-f55f-4126-9d38-dedb19577f4e"/>

### Pandas/ベクトル化されたUDF (Pandas/Vectorized UDFs)

Spark 2.3からは、Pythonで利用できるPandas UDFがあり、UDFの効率を向上させることができます。PandasのUDFは、Apache Arrowを利用して計算を高速化します。それが処理時間の改善にどう役立つかを見てみましょう。

* <a href="https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html" target="_blank">ブログ記事</a>
* <a href="https://spark.apache.org/docs/latest/sql-programming-guide.html#pyspark-usage-guide-for-pandas-with-apache-arrow" target="_blank">ドキュメンテーション</a>

<img src="https://databricks.com/wp-content/uploads/2017/10/image1-4.png" alt="Benchmark" width ="500" height="1500">

ユーザー定義関数が実行されます。
* <a href="https://arrow.apache.org/" target="_blank">Apache Arrow</a>は、Sparkで使用されてJVM と Python プロセス間のデータをほぼゼロの（デ）シリアライズコストで効率的に転送するためのインメモリ列型データ形式です。詳しくは<a href="https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html" target="_blank">こちら</a>をご覧ください。
* pandasのインスタンスおよびAPIと連携するため、関数内部でpandasを使用します。

**注**:Spark 3.0では、Pythonのタイプヒントを使用してPandas UDFを定義する必要があります。

In [0]:
from pyspark.sql.functions import pandas_udf

@pandas_udf("double")
def predict(*args: pd.Series) -> pd.Series:
    model_path = f"runs:/{run.info.run_id}/model" 
    model = mlflow.sklearn.load_model(model_path) # Load model
    pdf = pd.concat(args, axis=1)
    return pd.Series(model.predict(pdf))

prediction_df = spark_df.withColumn("prediction", predict(*spark_df.columns))
display(prediction_df)

<i18n value="e97526c6-ef40-4d55-9763-ee3ebe846096"/>

### Pandas Scalar Iterator UDF

モデルが非常に大きい場合、同じPythonワーカープロセスでバッチごとに同じモデルを繰り返しロードすることは、Pandas UDFにとって高いオーバーヘッドとなります。Spark 3.0では、Pandas UDFはpandas.Seriesまたはpandas.DataFrameのiteratorを受け取ることができるので、iterator内のシリーズごとにモデルを読み込むのではなく、一度だけモデルを読み込むことで済みます。

そうすれば、必要なセットアップのコストが発生する回数も少なくなります。扱うレコード数が **`spark.conf.get('spark.sql.execution.arrow.maxRecordsPerBatch')`** (デフォルトは 10,000) より多い場合、pandas scalar UDFはpd.Seriesのバッチを反復処理するので、スピードアップが見られるはずです。

一般的な構文：

```
@pandas_udf(...)
def predict(iterator):
    model = ... # load model
    for features in iterator:
        yield model.predict(features)
```

In [0]:
from typing import Iterator, Tuple

@pandas_udf("double")
def predict(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.Series]:
    model_path = f"runs:/{run.info.run_id}/model" 
    model = mlflow.sklearn.load_model(model_path) # Load model
    for features in iterator:
        pdf = pd.concat(features, axis=1)
        yield pd.Series(model.predict(pdf))

prediction_df = spark_df.withColumn("prediction", predict(*spark_df.columns))
display(prediction_df)

<i18n value="23b8296e-e0bc-481e-bd35-4048d532c71d"/>

### Pandas Function API

Pandas UDFを使う代わりに、Pandas Function APIを使うことができます。Apache Spark 3.0のこの新しい機能では、PySpark DataFrameに対してPandasインスタンスを取得・出力するPythonネイティブ関数を直接適用することができるようになりました。Apache Spark 3.0でサポートされるPandas Functions APIは、grouped map、mapとco-grouped mapです。

**`mapInPandas()`** は pandas.DataFrame のiteratorを入力とし、別の pandas.DataFrame のiteratorを出力する。モデルが入力として全てのカラムを必要とする場合、柔軟で使いやすいですが、DataFrame全体のシリアライズ/デシリアライズが必要です（入力として渡されるため）。iteratorが出力する各pandas.DataFrameのバッチサイズは、 **`spark.sql.execution.arrow.maxRecordsPerBatch`** の設定により制御できます。

In [0]:
def predict(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    model_path = f"runs:/{run.info.run_id}/model" 
    model = mlflow.sklearn.load_model(model_path) # Load model
    for features in iterator:
        yield pd.concat([features, pd.Series(model.predict(features), name="prediction")], axis=1)
    
display(spark_df.mapInPandas(predict, """`host_total_listings_count` DOUBLE,`neighbourhood_cleansed` BIGINT,`latitude` DOUBLE,`longitude` DOUBLE,`property_type` BIGINT,`room_type` BIGINT,`accommodates` DOUBLE,`bathrooms` DOUBLE,`bedrooms` DOUBLE,`beds` DOUBLE,`bed_type` BIGINT,`minimum_nights` DOUBLE,`number_of_reviews` DOUBLE,`review_scores_rating` DOUBLE,`review_scores_accuracy` DOUBLE,`review_scores_cleanliness` DOUBLE,`review_scores_checkin` DOUBLE,`review_scores_communication` DOUBLE,`review_scores_location` DOUBLE,`review_scores_value` DOUBLE, `prediction` DOUBLE"""))

<i18n value="d13b87a7-0625-4acc-88dc-438cf06e18bd"/>

あるいは、以下のようなスキーマを定義することもできます。

In [0]:
from pyspark.sql.functions import lit
from pyspark.sql.types import DoubleType

schema = spark_df.withColumn("prediction", lit(None).cast(DoubleType())).schema
display(spark_df.mapInPandas(predict, schema))

&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>